In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install -q scipy



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
#model_id = 'bigscience/bloom'
model_id = 'tiiuae/falcon-7b'

In [3]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map='auto', trust_remote_code=True)

Loading tiiuae/falcon-7b requires to execute some code in that repo, you can inspect the content of the repository at https://hf.co/tiiuae/falcon-7b. You can dismiss this prompt by passing `trust_remote_code=True`.
Do you accept? [y/N]  y


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading tiiuae/falcon-7b requires to execute some code in that repo, you can inspect the content of the repository at https://hf.co/tiiuae/falcon-7b. You can dismiss this prompt by passing `trust_remote_code=True`.
Do you accept? [y/N]  y


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [6]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [7]:
from datasets import load_dataset

data = load_dataset("thanhnew2001/alpaca_vn")

data = data.map(
    lambda samples: {
        "input_text": ' ### Human: ' + samples["question"] + ' ### Assisstant: ' + samples["answer"] + ' ### '
    }
, remove_columns= ['question', 'answer'])

data = data.map(lambda examples: tokenizer(examples["input_text"]), batched=True)


#print(data['train'])
#data['train'][0:5]

#data = data.map(lambda x: tokenizer(f"{x['question']}"), batched=True)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/31448 [00:00<?, ? examples/s]

Map:   0%|          | 0/31448 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2352 > 2048). Running this sequence through the model will result in indexing errors


In [8]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=10,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.741500
2,1.552100
3,1.597000
4,1.497300
5,1.570500
6,1.373300
7,1.869900
8,1.829900
9,1.436200
10,2.060000


TrainOutput(global_step=10, training_loss=1.7527698636054994, metrics={'train_runtime': 27.7323, 'train_samples_per_second': 1.442, 'train_steps_per_second': 0.361, 'total_flos': 286729489756416.0, 'train_loss': 1.7527698636054994, 'epoch': 0.0})

In [9]:
trainer.save_model('adapter-model')


In [11]:
from peft import PeftModel    

base_model = AutoModelForCausalLM.from_pretrained(
        model_id,
        return_dict=True,
        torch_dtype=torch.float16,
        trust_remote_code=True
)

model = PeftModel.from_pretrained(base_model, 'adapter-model').to('cuda')
print(f"Running merge_and_unload")
model = model.merge_and_unload() 
tokenizer = AutoTokenizer.from_pretrained(model_id)

model.save_pretrained("merged")
tokenizer.save_pretrained("merged")


Loading tiiuae/falcon-7b requires to execute some code in that repo, you can inspect the content of the repository at https://hf.co/tiiuae/falcon-7b. You can dismiss this prompt by passing `trust_remote_code=True`.
Do you accept? [y/N]  y


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Running merge_and_unload


('merged/tokenizer_config.json',
 'merged/special_tokens_map.json',
 'merged/tokenizer.json')

In [12]:
double_quant_nf4_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_use_double_quant=True
)
model_id ='merged'
model_double = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=double_quant_nf4_config, device_map="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading merged requires to execute some code in that repo, you can inspect the content of the repository at https://hf.co/merged. You can dismiss this prompt by passing `trust_remote_code=True`.
Do you accept? [y/N]  y


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
text = "Không có gì quý hơn độc lập tự do"
inputs = tokenizer(text, return_tensors="pt", return_token_type_ids=False).to("cuda:0")
outputs = model_double.generate(**inputs, max_new_tokens=200, pad_token_id=50256)
print(outputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

tensor([[45543,  5525,   904, 16210,   123,   294,  8533,   453,   137,   133,
           278,   140,   106,    89, 54188, 19768,   231,    78,   282, 28624,
           239,    91,   240, 19768,   121,   441,   253, 19768, 13739,   266,
          8533, 61447,    25,   193,    63,  5525,    84, 54188,  2583,   240,
         19768,   116,   904,   287,    83, 19768,   119,   904, 23250,  2575,
           100, 54188, 19768,   209,   240,  8533,    88, 54188,   140,   120,
         19768,   108,    78,   266, 19768,   231,    95, 23250,   140,   120,
         19768,   235,    84, 54188, 19768,   209]], device='cuda:0')
Không có gì quý hơn độc lập tự do của mình.
Tôi đã từng những ngày để tìm được một người để
